In [111]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timezone, timedelta
from zoneinfo import ZoneInfo
import re
import numpy as np

In [201]:
tal = pd.read_csv('episode_info', index_col=False)
#tal.drop("Unnamed: 0", axis=1, inplace=True)
#tal.drop_duplicates(inplace=True) 

In [202]:
tal.head()

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness


In [4]:
#tal.to_csv('episode_info', index=False)

In [5]:
tal.isna().sum()

ep_num            0
ep_name           0
pub_date          0
host              0
contributers      0
num_acts          0
last_timestamp    0
url_suffix        0
dtype: int64

In [295]:
import string
s = "hi / i\\'m do'ing good. !"
translator = str.maketrans('','',string.punctuation)
clean_text = s.translate(translator)
meow = clean_text
re.sub(' +',' ',meow).strip().replace(' ','-')

'hi-im-doing-good'

In [300]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [314]:
def get_url_suffix(name):
    translator = str.maketrans("","","!\"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~’‘")
    clean = name.translate(translator)
    print(clean)
    return re.sub(" +"," ", clean).strip().replace(" ","-")

In [315]:
get_url_suffix("What You Lookin’ At?")

What You Lookin At


'What-You-Lookin-At'

In [316]:
get_url_suffix("Valentine’s Day ’99")

Valentines Day 99


'Valentines-Day-99'

In [317]:
tal.apply(lambda row: get_url_suffix(row['ep_name']), axis=1)

New Beginnings
Small Scale Sin
Poultry Slam 1995
Vacations
Anger and Forgiveness
Christmas
Quitting
New Year
Julia Sweeney
Double Lives
Enemies
Animals
Love
Accidental Documentaries
Dawn
Economy
Name Change  No Theme
Liars
Rich Guys
From A Distance
Factions
Adult Children
Drama Bug
Teenaged Girls
Basketball
Fathers Day 96
The Cruelty of Children
Detectives
Bob Dole
Obsession
When You Talk About Music
Republican Convention
A Night at the Wiener Circle
Democratic Convention
Fall Clearance Sale
Letters
The Job That Takes Over Your Life
Simulated Worlds
Halloween
Lessons
Politics
Get Over It
Faustian Bargains
Poultry Slam 1996
Media Fringe
Sissies
Christmas and Commerce
Justice
Animals
Shoulda Been Dead
Animals Die People Ponder
Edge of Sanity
Valentines Day 97
Sinatra
Three Women and the Sex Industry
Name Change
Delivery
Small Towns
Fire
Business of Death
Fiasco 1997
Something for Nothing
One Thing
Summer
Whos Canadian
Tales from the Net
Your Dream My Nightmare
Lincolns Second Inaugural
D

0                                         New-Beginnings
1                                        Small-Scale-Sin
2                                      Poultry-Slam-1995
3                                              Vacations
4                                  Anger-and-Forgiveness
                             ...                        
845                       The-Official-Unofficial-Record
846                                         The-Narrator
847    If-You-Want-to-Destroy-My-Sweater-Hold-This-Th...
848                              Try-a-Little-Tenderness
849                                          Pivot-Point
Length: 850, dtype: object

In [318]:
tal['url_suffix'] = tal.apply(lambda row: str(row['ep_num']) + '/' + get_url_suffix(row['ep_name']), axis=1)
tal.head()

New Beginnings
Small Scale Sin
Poultry Slam 1995
Vacations
Anger and Forgiveness
Christmas
Quitting
New Year
Julia Sweeney
Double Lives
Enemies
Animals
Love
Accidental Documentaries
Dawn
Economy
Name Change  No Theme
Liars
Rich Guys
From A Distance
Factions
Adult Children
Drama Bug
Teenaged Girls
Basketball
Fathers Day 96
The Cruelty of Children
Detectives
Bob Dole
Obsession
When You Talk About Music
Republican Convention
A Night at the Wiener Circle
Democratic Convention
Fall Clearance Sale
Letters
The Job That Takes Over Your Life
Simulated Worlds
Halloween
Lessons
Politics
Get Over It
Faustian Bargains
Poultry Slam 1996
Media Fringe
Sissies
Christmas and Commerce
Justice
Animals
Shoulda Been Dead
Animals Die People Ponder
Edge of Sanity
Valentines Day 97
Sinatra
Three Women and the Sex Industry
Name Change
Delivery
Small Towns
Fire
Business of Death
Fiasco 1997
Something for Nothing
One Thing
Summer
Whos Canadian
Tales from the Net
Your Dream My Nightmare
Lincolns Second Inaugural
D

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness


### Let's work on data validation from the episode pages now that we have the episode number and name

for i in np.arange(1,853):
    ep_number = str(i)
    transcript_url = 'https://www.thisamericanlife.org/'+ ep_number + '/transcript'
    page = requests.get(transcript_url)
    with open('transcript-html/'+ep_number+'.html','wb+') as f: 
        f.write(page.content)

In [ ]:
domain = 'https://www.thisamericanlife.org/'
for ep in tal.itertuples():
    url = domain + ep[8]
    ep_num = ep[1]
    page = requests.get(url)
    with open('episode-html/'+str(ep_num)+'.html','wb+') as f:
        f.write(page.content)
    f.close()
    #print(url, ep_num)

In [7]:
tal

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness
...,...,...,...,...,...,...,...,...
845,848,The Official Unofficial Record,2024-11-23 08:50:53-08:00,Ira Glass,"['Lilly Sullivan', 'Nancy Updike', 'Ira Glass']",3,0 days 00:59:44.810000,848/The-Official-Unofficial-Record
846,849,The Narrator,2024-12-14 03:40:47-08:00,Chana Joffe-Walt,['Chana Joffe-Walt'],2,0 days 00:55:52.570000,849/The-Narrator
847,850,"If You Want to Destroy My Sweater, Hold This T...",2024-12-21 02:30:34-08:00,Ira Glass,"['Lilly Sullivan', 'Mike Comite', 'Diane Wu', ...",3,0 days 01:02:52.080000,"850/If-You-Want-to-Destroy-My-Sweater,-Hold-Th..."
848,851,Try a Little Tenderness,2025-01-13 07:30:23-08:00,Ira Glass,"['Etgar Keret', 'Aviva DeKornfeld', 'Ira Glass']",4,0 days 00:58:06.240000,851/Try-a-Little-Tenderness


In [14]:
test = open('episode-html/30.html', 'r')
soup = BeautifulSoup(test, 'html.parser')
date = soup.find("meta", property="article:published_time")['content']
dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
dt#.astimezone(ZoneInfo('US/Pacific'))

datetime.datetime(1996, 7, 26, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [238]:
def get_episode(soup): 
    try:
        return re.split(" - ", soup.title.text)[0]
    except Exception as e: 
        print(e)
        return None
        
def get_date(soup): 
    try:
        date = soup.find("meta", property="article:published_time")['content']
        dt = datetime.strptime(date, "%Y-%m-%dT%H:%M:%S%z")
        return dt
    except Exception as e: 
        print(e)
        return None

In [239]:
get_episode(soup)

'Obsession'

In [242]:
get_date(soup)

datetime.datetime(1996, 7, 26, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=72000)))

In [243]:
soup.find_all("div", class_='field field-name-field-contributor field-type-entityreference field-label-above')

[<div class="field field-name-field-contributor field-type-entityreference field-label-above"><h2 class="field-label">By</h2><div class="field-items"><div class="field-item even"><a href="/archive?contributor=8731">Ira Glass</a></div><div class="field-item odd"><a href="/archive?contributor=9085">Paul Tough</a></div></div></div>,
 <div class="field field-name-field-contributor field-type-entityreference field-label-above"><h2 class="field-label">By</h2><div class="field-items"><div class="field-item even"><a href="/archive?contributor=9045">Lauren Slater</a></div><div class="field-item odd"><a href="/archive?contributor=9085">Paul Tough</a></div></div></div>]

In [244]:
def get_contributors(soup):
    try:
        contributors = set()
        c_list = soup.find_all("div", class_='field field-name-field-contributor field-type-entityreference field-label-above')
        for i in c_list:
            for j in i.find_all('a'):
                contributors.add(j.text)
        return list(contributors)
    except Exception as e:
        return None


get_contributors(soup)



['Paul Tough', 'Lauren Slater', 'Ira Glass']

In [245]:
def get_acts(soup, ep_num):
    act_names = []
    r_href = re.compile(str(ep_num))
    try:
        for element in soup.find_all('a', class_='goto goto-act'):
            if re.search(r_href, str(element)):
                act_names.append(element.text)
        return act_names
    except Exception as e:
        return None

In [246]:
get_acts(soup, 30)

['Act One', 'Further Out', 'Sacred Versus Profane', 'Heroic Obsessions']

In [247]:
meow = soup.find_all("a", class_="goto goto-act", href="30")
meow

[]

In [248]:
tags = [tag.name for tag in soup.find_all()]
class_list = set()
for tag in tags: 
    for i in soup.find_all(tag):
        if i.has_attr("class"):
            if len(i['class']) !=0:
                class_list.add(" ".join(i['class']))

In [249]:
class_list

{'about has-children',
 'act-header',
 'actions',
 'acts',
 'apple',
 'block block-system',
 'body',
 'bottom',
 'button form-submit',
 'clear',
 'close',
 'close ignore',
 'collection',
 'column sidebar',
 'contact',
 'container clearfix',
 'content',
 'contributor',
 'copyright',
 'credit',
 'cta_link',
 'current',
 'date-display-single',
 'description',
 'desktop',
 'download',
 'element-invisible',
 'element-invisible element-focusable',
 'episode',
 'episode-header',
 'episode-title',
 'episodes',
 'facebook',
 'field field-name-body field-type-text-with-summary field-label-hidden',
 'field field-name-field-act-label field-type-text field-label-hidden',
 'field field-name-field-acts field-type-entityreference field-label-hidden',
 'field field-name-field-contributor field-type-entityreference field-label-above',
 'field field-name-field-episode-number',
 'field field-name-field-episode-number field-type-number-integer field-label-hidden',
 'field field-name-field-radio-air-date',


In [262]:
f = open('episode-html/17.html','r')
content = f.read()
s = BeautifulSoup(content, 'html.parser')
get_episode(s)
#get_contributers(s)
#get_date(s)
#get_acts(s, 17)

'Page not found'

In [283]:
max_ep = 852
data = []

for ep in np.arange(1,max_ep+1):
    try:
        with open('episode-html/' + str(ep) + '.html', 'r') as f:
            print(ep)
            content = f.read()
            s = BeautifulSoup(content, 'html.parser')
            name = get_episode(s)
            c = get_contributers(s)
            date = get_date(s)
            acts = get_acts(s, ep)
            data.append([ep, name, date, c, acts])
    except Exception as e:
        print(str(e) + " at episode " + str(ep))
    finally:
        f.close()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
'NoneType' object is not subscriptable
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
'NoneType' object is not subscriptable
94
95
96
97
98
99
100
101
102
103
104
105
106
'NoneType' object is not subscriptable
107
108
109
110
111
112
113
114
115
116
117
118
'NoneType' object is not subscriptable
119
120
121
122
'NoneType' object is not subscriptable
123
124
125
126
'NoneType' object is not subscriptable
127
128
129
130
131
132
'NoneType' object is not subscriptable
133
134
'NoneType' object is not subscriptable
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
'NoneType' object is not subscriptable
160
161
162
163
'NoneType' object is not subscriptable
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
18

KeyboardInterrupt: 

In [285]:
tal.iloc[117]

ep_num                                                          118
ep_name                                        What You Lookin’ At?
pub_date                                  2017-12-14 10:24:29-08:00
host                                                      Ira Glass
contributers      ['David Rakoff', 'Announcer', 'Sarah Vowell', ...
num_acts                                                          3
last_timestamp                               0 days 00:58:55.700000
url_suffix                                  118/What-You-Lookin’-At
Name: 117, dtype: object

In [236]:
temp = pd.DataFrame(data, columns = ['num', 'name', 'pub_date', 'contributors', 'acts'])
temp[1:10]

,num,name,pub_date,contributors,acts
1,2,Small Scale Sin,1995-11-24 00:00:00-05:00,"[Jeff Dorchen, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act On..."
2,3,Poultry Slam 1995,1995-12-01 00:00:00-05:00,"[Julie Showalter, David Sedaris, Ira Glass]","[Act One, Act Two, Act Three, Act Four, Act Fi..."
3,4,Vacations,1995-12-08 00:00:00-05:00,"[Sandra Loh, Ira Glass, David Sedaris]","[Act One, Act Two, Act Three]"
4,5,Anger and Forgiveness,1995-12-15 00:00:00-05:00,"[Iris Moore, Larry Steeger, Jack Hitt, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act Fo..."
5,6,Christmas,1995-12-22 00:00:00-05:00,"[The Pinetree Gang, Beau O'Reilly, Ira Glass, ...","[Prologue, Act One, Act Two, Act Three, Act Tw..."
6,7,Quitting,1995-12-29 00:00:00-05:00,"[Lisa Buscani, Sandra Loh, Ira Glass]","[Prologue, Act One, Act Two, Act Three, Act Fo..."
7,8,New Year,1996-01-03 00:00:00-05:00,[Ira Glass],"[Prologue, Act One, Act Two, Act Three]"
8,9,Julia Sweeney,1996-01-10 00:00:00-05:00,"[Julia Sweeney, Ira Glass]","[Prologue, Julia's Brother Gets Cancer, Julia ..."
9,10,Double Lives,1996-01-17 00:00:00-05:00,"[Ira Glass, Neo-futurists]","[Prologue, The Book Tour, Dad In The Closet, L..."


In [237]:
temp.tail()

,num,name,pub_date,contributors,acts
847,848,The Official Unofficial Record,2024-11-15 13:32:32-05:00,[],"[Prologue, Best Acta in a Dramatic Role, Meanw..."
848,849,The Narrator,2024-12-06 10:48:37-05:00,[],"[Prologue, Part One, Part Two]"
849,850,Page not found,None,[],[]
850,851,Try a Little Tenderness,2025-01-03 10:33:26-05:00,[],"[Prologue, The Big Nap, The Gladiator Starring..."
851,852,Pivot Point,2025-01-10 11:47:17-05:00,[],"[Prologue, Who’s Laughing Now?, The View From ..."


In [225]:
tal

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness
...,...,...,...,...,...,...,...,...
845,848,The Official Unofficial Record,2024-11-23 08:50:53-08:00,Ira Glass,"['Lilly Sullivan', 'Nancy Updike', 'Ira Glass']",3,0 days 00:59:44.810000,848/The-Official-Unofficial-Record
846,849,The Narrator,2024-12-14 03:40:47-08:00,Chana Joffe-Walt,['Chana Joffe-Walt'],2,0 days 00:55:52.570000,849/The-Narrator
847,850,"If You Want to Destroy My Sweater, Hold This T...",2024-12-21 02:30:34-08:00,Ira Glass,"['Lilly Sullivan', 'Mike Comite', 'Diane Wu', ...",3,0 days 01:02:52.080000,850/If-You-Want-to-Destroy-My-Sweater-Hold-Thi...
848,851,Try a Little Tenderness,2025-01-13 07:30:23-08:00,Ira Glass,"['Etgar Keret', 'Aviva DeKornfeld', 'Ira Glass']",4,0 days 00:58:06.240000,851/Try-a-Little-Tenderness


In [214]:
a = temp['num'].to_numpy()

In [217]:
tal[tal['ep_num'].isin(a)]

,ep_num,ep_name,pub_date,host,contributers,num_acts,last_timestamp,url_suffix
0,1,New Beginnings,2017-12-14 10:24:29-08:00,Ira Glass,"['Kevin Kelly', 'Lawrence Steger', 'Ira Glass']",4,0 days 01:01:50.290000,1/New-Beginnings
1,2,Small Scale Sin,2017-12-14 10:24:29-08:00,Ira Glass,"['Announcer', 'Ira Glass']",3,0 days 01:01:13.840000,2/Small-Scale-Sin
2,3,Poultry Slam 1995,2017-12-14 10:24:29-08:00,Ira Glass,"['David Sedaris', 'Chickenman Announcer', 'Ira...",4,0 days 00:58:57.870000,3/Poultry-Slam-1995
3,4,Vacations,2017-12-14 10:24:29-08:00,Ira Glass,"['Sandra Tsing Loh', 'Ira Glass', 'David Sedar...",3,0 days 00:58:42.070000,4/Vacations
4,5,Anger and Forgiveness,2017-12-14 10:24:29-08:00,Ira Glass,"['Glen Fitzgerald', 'Jack Hitt', 'Cheryl Trykv...",5,0 days 01:00:33.100000,5/Anger-and-Forgiveness
...,...,...,...,...,...,...,...,...
845,848,The Official Unofficial Record,2024-11-23 08:50:53-08:00,Ira Glass,"['Lilly Sullivan', 'Nancy Updike', 'Ira Glass']",3,0 days 00:59:44.810000,848/The-Official-Unofficial-Record
846,849,The Narrator,2024-12-14 03:40:47-08:00,Chana Joffe-Walt,['Chana Joffe-Walt'],2,0 days 00:55:52.570000,849/The-Narrator
847,850,"If You Want to Destroy My Sweater, Hold This T...",2024-12-21 02:30:34-08:00,Ira Glass,"['Lilly Sullivan', 'Mike Comite', 'Diane Wu', ...",3,0 days 01:02:52.080000,"850/If-You-Want-to-Destroy-My-Sweater,-Hold-Th..."
848,851,Try a Little Tenderness,2025-01-13 07:30:23-08:00,Ira Glass,"['Etgar Keret', 'Aviva DeKornfeld', 'Ira Glass']",4,0 days 00:58:06.240000,851/Try-a-Little-Tenderness
